In [ ]:
from PIL import Image
import os
from concurrent.futures import ThreadPoolExecutor

PATH = '/path/to/folder'  # Change path here

def augment_image(file_path, folder_path):
    # Open the original image
    with Image.open(file_path) as img:
        # Get the file name without the extension
        base_name = os.path.splitext(os.path.basename(file_path))[0]

        # List to store all image transformations (original, rotated, flipped)
        images = []

        # Original image
        images.append((img, "original"))

        # Rotate the image by 90, 180, and 270 degrees
        images.append((img.rotate(90), "rot90"))
        images.append((img.rotate(180), "rot180"))
        images.append((img.rotate(270), "rot270"))

        # Create horizontal flips for all variations (original + rotated)
        for image, suffix in images:
            # Save the original and its rotated variants
            image.save(os.path.join(folder_path, f"{base_name}_{suffix}.png"))

            # Flip horizontally
            flipped = image.transpose(Image.FLIP_LEFT_RIGHT)
            flipped.save(os.path.join(folder_path, f"{base_name}_{suffix}_flipped.png"))

    # Remove the original image after augmentation
    os.remove(file_path)

def augment_images_in_folder_parallel(folder_path):
    # List all PNG files in the folder
    file_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(".png")]

    # Use ThreadPoolExecutor to process images in parallel
    with ThreadPoolExecutor() as executor:
        # Submit tasks for parallel execution
        futures = [executor.submit(augment_image, file_path, folder_path) for file_path in file_paths]
        
        # Wait for all threads to complete and print status
        for future in futures:
            future.result()  # Blocks until each future is done

if __name__ == "__main__":

    # Run the augmentation function in parallel
    augment_images_in_folder_parallel(PATH)
